In [22]:
import numpy as np
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from wordcloud import WordCloud
from textblob import Word

import matplotlib.pyplot as plt

In [23]:
df = pd.read_csv("WTT_offers.csv")

df = df[["name", "description"]]
df

,name,description
0,Manager - Data & Analytics Engineering,The Data team at Welcome to the Jungle: Part o...
1,Margo Analytics - Data Engineer - H/F,Margo Analytics est l'entité experte de Margo...
2,Data Analyst,Être Data Analyst chez Wewyse c’est : Intégr...
3,Data Analyst - Stage de 6 mois,Afin de mieux comprendre nos clients et leurs ...
4,Data Analyst - Stage - Paris,Travailler chez papernest : définition. Cer...
...,...,...
57,DATA ANALYST EXPERT POWER BI,En raison de l'augmentation de l'appétences de...
58,DATA ANALYST MARKETING DIGITAL F/H,REJOIGNEZ UN COLLECTIF ANIME PAR LE GOUT DU DE...
59,Data Analyst Appui au Pilotage (F/H),Contexte Vous souhaitez rejoindre une entrepri...
60,Sénior data scientist/analyst,Intégré(e) en tant que collaborateur et data s...


In [4]:
df["description"] = df["description"].apply(lambda x: " ".join(x.lower() for x in x.split()))
df["description"] = df["description"].str.replace('[^\w\s]', ' ')
# df["description"] = df["description"].str.replace('\d+', '')
stop = stopwords.words("french")
df["description"] = df["description"].apply(lambda x: ' '.join(x for x in x.split() if x not in stop))
df["description"] = df["description"].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

df.head()

C:\Users\royde\AppData\Local\Temp\ipykernel_7160\1772313848.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["description"] = df["description"].str.replace('[^\w\s]', ' ')


,name,description
0,Manager - Data & Analytics Engineering,the data team at welcome to the jungle part of...
1,Margo Analytics - Data Engineer - H/F,margo analytics entité experte margo group pro...
2,Data Analyst,être data analyst chez wewyse intégrer communa...
3,Data Analyst - Stage de 6 mois,afin mieux comprendre client leurs besoins aus...
4,Data Analyst - Stage - Paris,travailler chez papernest définition certains ...


In [5]:
df["name"] = df["name"].apply(lambda x: " ".join(x.lower() for x in x.split()))
df["name"] = df["name"].str.replace('[^\w\s]', ' ')
stop = stopwords.words("french")
df["name"] = df["name"].apply(lambda x: ' '.join(x for x in x.split() if x not in stop))
df["name"] = df["name"].apply(lambda x: ' '.join([Word(word).lemmatize() for word in x.split()]))
df["name"] = df["name"].str.replace('h', '').str.replace('f', '')
df.head()

C:\Users\royde\AppData\Local\Temp\ipykernel_7160\3600074061.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["name"] = df["name"].str.replace('[^\w\s]', ' ')


,name,description
0,manager data analytics engineering,the data team at welcome to the jungle part of...
1,margo analytics data engineer,margo analytics entité experte margo group pro...
2,data analyst,être data analyst chez wewyse intégrer communa...
3,data analyst stage 6 mois,afin mieux comprendre client leurs besoins aus...
4,data analyst stage paris,travailler chez papernest définition certains ...


In [6]:
df["name"][df["name"].str.contains("engineer")] = "Data Engineer"
df["name"][df["name"].str.contains("analyst")] = "Data Analyst"
df["name"][df["name"].str.contains("scientist")] = "Data Scientist"

In [24]:
df

,name,description
0,Manager - Data & Analytics Engineering,The Data team at Welcome to the Jungle: Part o...
1,Margo Analytics - Data Engineer - H/F,Margo Analytics est l'entité experte de Margo...
2,Data Analyst,Être Data Analyst chez Wewyse c’est : Intégr...
3,Data Analyst - Stage de 6 mois,Afin de mieux comprendre nos clients et leurs ...
4,Data Analyst - Stage - Paris,Travailler chez papernest : définition. Cer...
...,...,...
57,DATA ANALYST EXPERT POWER BI,En raison de l'augmentation de l'appétences de...
58,DATA ANALYST MARKETING DIGITAL F/H,REJOIGNEZ UN COLLECTIF ANIME PAR LE GOUT DU DE...
59,Data Analyst Appui au Pilotage (F/H),Contexte Vous souhaitez rejoindre une entrepri...
60,Sénior data scientist/analyst,Intégré(e) en tant que collaborateur et data s...


In [31]:
technical_skills = [
    'python',
    'flask',
    'pandas',
    'spark',
    'scikit-learn',
    'numpy',
    'sql',
    'mysql',
    'nltk',
    'fastapi',
    'pytorch',
    'snowflake',
    'pandas',
    'rivery',
    'django',
    'react',
    'html',
    'machine learning',
    'tableau',
    'bi',
    'powerbi',
    'looker',
    'data warehouse',
    'dbt',
    'power bi',
    'ia',
    'ai'
]

def extract_tech_skills(description):
    skills = set(description.lower().split())
    extracted_skills = list(skills.intersection(technical_skills))
    return extracted_skills

df["tech_skills"] = df["description"].apply(extract_tech_skills)


df

,name,description,tech_skills
0,Manager - Data & Analytics Engineering,The Data team at Welcome to the Jungle: Part o...,"[ia, looker]"
1,Margo Analytics - Data Engineer - H/F,Margo Analytics est l'entité experte de Margo...,[ia]
2,Data Analyst,Être Data Analyst chez Wewyse c’est : Intégr...,[]
3,Data Analyst - Stage de 6 mois,Afin de mieux comprendre nos clients et leurs ...,"[looker, bi, dbt]"
4,Data Analyst - Stage - Paris,Travailler chez papernest : définition. Cer...,[sql]
...,...,...,...
57,DATA ANALYST EXPERT POWER BI,En raison de l'augmentation de l'appétences de...,[powerbi]
58,DATA ANALYST MARKETING DIGITAL F/H,REJOIGNEZ UN COLLECTIF ANIME PAR LE GOUT DU DE...,[]
59,Data Analyst Appui au Pilotage (F/H),Contexte Vous souhaitez rejoindre une entrepri...,[bi]
60,Sénior data scientist/analyst,Intégré(e) en tant que collaborateur et data s...,[]


In [32]:
df["description"].iloc[61]

'Nous travaillons avec  +400 startups et scale ups  de la scène tech française et européenne, à la recherche de Data Analysts expérimentés  de Confirmés à C-Level : Heetch, Scaleway, Skello, Blablacar, Sendinblue, Mirakl, Partoo, Frichti, Swan, Jellysmack, MangoPay, Coinhouse … mais aussi les startups d’eFounders comme Collective Work, Numeral, ou Folk… et bien d’autres. Nous développons une communauté de hauts talents avec pour vocation d’apporter du soutien à leur carrière tout au long de leur vie professionnelle. Événements, ressources, sessions de recrutement…'